## 多输入通道

In [1]:
import torch
from d2l import torch as d2l

def corr2d_multi_in(X, K):
    # 先遍历“X”和“K”的第0个维度（通道维度），再把它们加在一起
    return sum(d2l.corr2d(x, k) for x, k in zip(X, K))

In [22]:
X = torch.tensor([[[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]],
                  [[1.0, 2.0, 3.0], [4.0, 5.0, 6.0], [7.0, 8.0, 9.0]]])
K = torch.tensor([[[0.0, 1.0], [2.0, 3.0]], [[1.0, 2.0], [3.0, 4.0]]])

print(f'X.shape={X.shape}\nK.shape={K.shape}')
corr2d_multi_in(X, K)

X.shape=torch.Size([2, 3, 3])
K.shape=torch.Size([2, 2, 2])


tensor([[ 56.,  72.],
        [104., 120.]])

## 多输出通道

In [26]:
def corr2d_multi_in_out(X, K):
    # 迭代“K”的第0个维度，每次都对输入“X”执行互相关运算。
    # 最后将所有结果都叠加在一起
    return torch.stack([corr2d_multi_in(X, k) for k in K], 0)

In [24]:
K = torch.stack((K, K + 1, K + 2), 0)
K.shape

torch.Size([3, 2, 2, 2])

In [27]:
corr2d_multi_in_out(X, K)

tensor([[[ 56.,  72.],
         [104., 120.]],

        [[ 76., 100.],
         [148., 172.]],

        [[ 96., 128.],
         [192., 224.]]])

## $1 \times 1$卷积层

In [28]:
def corr2d_multi_in_out_1x1(X, K):
    c_i, h, w = X.shape
    c_o = K.shape[0]
    X = X.reshape((c_i, h * w))
    K = K.reshape((c_o, c_i))
    # 全连接层中的矩阵乘法
    Y = torch.matmul(K, X)
    return Y.reshape((c_o, h, w))

In [29]:
X = torch.normal(0, 1, (3, 3, 3))
K = torch.normal(0, 1, (2, 3, 1, 1))

Y1 = corr2d_multi_in_out_1x1(X, K)
Y2 = corr2d_multi_in_out(X, K)
assert float(torch.abs(Y1 - Y2).sum()) < 1e-6

## 小结

- 多输入多输出通道可以用来扩展卷积层的模型。
- 当以每像素为基础应用时，1x1卷积层相当于全连接层。
- 1x1卷积层通常用于调整网络层的通道数量和控制模型复杂性。

## 练习

1. 假设我们有两个卷积核，大小分别为$K_1$和$k_2$（中间没有非线性激活函数）。
    1. 证明运算可以用单次卷积来表示。
    2. 这个等效的单个卷积核的维数是多少呢？
    3. 反之亦然吗？

2. 假设输入为$c_i \times h \times w$，卷积核大小为$c_o \times c_i \times k_h \times k_w$，填充为$(p_h, p_w)$，步幅为$s_h, s_w$。
    1. 前向传播的计算成本（乘法和加法）是多少？
    2. 内存占用是多少？
    3. 反向传播的内存占用是多少？
    4. 反向传播的计算成本是多少？
3. 如果我们将输入通道$c_i$和输出通道$c_o$的数量加倍，计算数量会增加多少？如果我们把填充数量翻一番会怎么样？
4. 如果卷积核的高度和宽度是$k_h = k_w = 1$，前向传播的计算复杂度是多少？
5. 本节最后一个示例中的变量Y1和Y2是否完全相同？为什么？
6. 当卷积窗口不是$1 \times 1$时，如何使用矩阵乘法实现卷积？